This was the latest version.  It was designed to keep collecting and saving the data and re-reading in order to handle situations like getting blocked midstream. But she did not get it working because she had problems taking into account a changed year range.


In [ ]:
import re
import os
import time
import random
import pandas as pd
from datetime import datetime
import shutil
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

# Path for Huang
chrome_driver_path = '/Users/huangshilin/Desktop/22 Summer/jstor scrape/chromedriver'
fileDest = '/Users/huangshilin/Desktop/22 Summer/econlit'
fileDest_backup = '/Users/huangshilin/Desktop/22 Summer/econlit/backup'


# Path for Murrell
#chrome_driver_path = "D:/corping2022/datahsl/otherFiles/chromedriver/chromedriver.exe"
#this was necessary for Murrell to give permission to open the driver on Windows
#os.chmod('D:/corping2022/datahsl/otherFiles/chromedriver/', 755)
#fileDest = 
#fileDest_backup = 

In [ ]:
def get_web_driver(): 
    s = Service(chrome_driver_path)
    driver = webdriver.Chrome(service = s)
    
    return driver

In [ ]:
# open the website
url = 'https://lib.guides.umd.edu/az.php?q=EconLit'
driver = get_web_driver()
driver.implicitly_wait(60)
driver.get(url)
driver.implicitly_wait(60)
# click EconLit
driver.find_element(By.CLASS_NAME,'s-lg-az-result-title').click()

In [ ]:
# This function chooses a random number and lets the program sleep for that amount of seconds.
# You need to put a start number, and the program will sleep for at least (start) seconds.
# You also need to put an incrementing number, and the program will sleep for at most (start + inc) seconds.
def my_sleep(start, inc):
    time.sleep(random.randint(start, start + inc + 1)) #seconds

In [ ]:
# log into UMD account, please put your Directory Id and Password here
directory_id = ''
password = ''
driver.find_element(By.ID,'username').send_keys(directory_id)
driver.implicitly_wait(60)
my_sleep(20, 10)
driver.find_element(By.ID,'password').send_keys(password)
driver.implicitly_wait(60)
my_sleep(20, 10)
driver.find_element(By.NAME,'_eventId_proceed').click()
driver.implicitly_wait(60)
my_sleep(20, 10)

In [ ]:
# accept DUO

In [ ]:
# click trust the website
driver.find_element(By.ID, 'trust-browser-button').click()
driver.implicitly_wait(60)

In [ ]:
# This function updates the excel.
def update_output(output_file_name, year_list, all_list, keyword_list, percentage_list):
    # put the results in an excel
    spreadsheet = pd.read_excel(os.path.join(fileDest, output_file_name))
    spreadsheet['Year'] = year_list
    spreadsheet['all'] = all_list
    spreadsheet[keyword] = keyword_list
    spreadsheet['Percentage'] = percentage_list
    #store the excel
    spreadsheet.to_excel(os.path.join(fileDest, output_file_name), index = False)

In [ ]:
def update_all(file_name, year_list, all_list):
    spreadsheet['Year'] = year_list
    spreadsheet['all'] = all_list
    spreadsheet.to_excel(os.path.join(fileDest, file_name), index = False)

In [ ]:
# return the number of articles in that year. If there is an error, return -2
def search_articles(year):
    try: 
        # input from year
        from_year = driver.find_element(By.CLASS_NAME,'filterDateFrom')
        from_year.clear()
        from_year.send_keys(year)
        driver.implicitly_wait(60)
        my_sleep(20, 10)
        # input to year
        to_year = driver.find_element(By.CLASS_NAME,'filterDateTo')
        to_year.clear()
        to_year.send_keys(year)
        driver.implicitly_wait(60)
        my_sleep(20, 10)
        # click search
        driver.find_element(By.NAME, 'SearchBtn').click()
        driver.implicitly_wait(60)
        my_sleep(20, 10)
        
        # extract the number of articles
        result = driver.find_element(By.ID,'ctl00_ctl00__bodyTag').text
        driver.implicitly_wait(60)
        my_sleep(20, 10)
        num = re.search('Search Results:.*of (.*)', result)
        num = int(num.group(1).replace(',', ''))
        print(num, "articles with keyword", keyword, "in", year)
        return num
        
    except: 
        print("error with the keyword", keyword, "in", year)
        return -2
    

First, let's collect the number of all articles in each year.

In [ ]:
# Set up the excel
start_year_all = 1996
ending_year_all = 2002
file_name = 'EconLit.xlsx'

# if file exists
if os.path.exists(fileDest+'/'+file_name):
    # duplicate the file and store it in a different folder
    shutil.copyfile(fileDest+'/'+file_name, fileDest_backup+'/'+file_name)
    
    # add date and time to the backup file's filename
    date = str(datetime.today().strftime("%m.%d.%y"))
    time = str(datetime.now().strftime("%H.%M.%S"))
    new_name = 'EconLit_' + date + '_' + time + '.xlsx'
    old_file = os.path.join(fileDest_backup, file_name)
    new_file = os.path.join(fileDest_backup, new_name)
    os.rename(old_file, new_file)
    
    # read the old file
    old_spreadsheet = pd.read_excel(os.path.join(fileDest, file_name))
    # a and b are the start year and end year in the old spreadsheet
    year = old_spreadsheet['Year'].tolist()
    #all_list = spreadsheet['all'].tolist()
    a = year[0]
    b = year[-1]
    
    year_list = []
    all_list = []
    # if we need a wider range of year
    if start_year_all < a:
        # put in default values
        for year in range (start_year_all, a):
            year_list.append(year)
            all_list.append(-1)
        
        #df1.concat(old_spreadsheet)
            #year_list.append(year)
            #all_list.append(-1)
            
    if ending_year_all > b:
        # put in default values
        for year in range (b + 1, ending_year_all + 1):
            year_list.append(year)
            all_list.append(-1)
            
    #create a new spreadsheet for the extra years
    if year_list:        
        df = pd.DataFrame({'Year': year_list,
                   'all': all_list})
        #print(df)
        new_spreadsheet = pd.concat([old_spreadsheet, df])
        new_spreadsheet = new_spreadsheet.sort_values(by = ['Year'])
        new_spreadsheet = new_spreadsheet.drop_duplicates(subset=['Year'])
        print(new_spreadsheet)
        new_spreadsheet.to_excel(os.path.join(fileDest, file_name), index = False)
        
    
    #update_all(file_name, year_list, all_list)
        
# if the file doesn't exist
else:
    # create an excel to store information
    writer = pd.ExcelWriter(os.path.join(fileDest, file_name))
    writer.save()
    # columns in the excel
    year_list = [] # year
    all_list = [] # number of all articles in that year

    # put in default values
    for year in range (start_year_all, ending_year_all + 1):
        year_list.append(year)
        all_list.append(-1)
    
    spreadsheet = pd.read_excel(os.path.join(fileDest,file_name))
    spreadsheet['Year'] = year_list
    spreadsheet['all'] = all_list
    
    #store the excel
    spreadsheet.to_excel(os.path.join(fileDest, file_name), index = False)

In [ ]:
# scrape the number of all articles

# put in any keyword
driver.find_element(By.NAME, 'GuidedSearchFormData[1].SearchTerm').send_keys('institutions')
driver.implicitly_wait(60)
my_sleep(20, 10)
# click search
driver.find_element(By.NAME, 'SearchBtn').click()
driver.implicitly_wait(60)
my_sleep(20, 10)
search_articles(2000)
# remove the keyword
driver.find_element(By.NAME, 'GuidedSearchFormData[1].SearchTerm').clear()
driver.implicitly_wait(60)
my_sleep(20, 10)
    # click search
driver.find_element(By.NAME, 'SearchBtn').click()
driver.implicitly_wait(60)
my_sleep(20, 10)
    # Note: we cannot search without a keyword, if we don't do a search with a keyword beforehand

#open the spreadsheet
spreadsheet = pd.read_excel(os.path.join(fileDest,file_name))
all_list = spreadsheet['all'].tolist()
year_list = spreadsheet['Year'].tolist()
print(all_list)
print(year_list)
for i in all_list:
    if i < 0:
        num = search_articles(year)
        idx = year_list.index(year)
        all_list[idx] = num
        update_all(file_name, year_list, all_list)



In [ ]:
# choose the year and the keyword
start_year = 2010
ending_year = 2022
keyword = 'institutions'
output_file_name = 'EconLit_' + keyword.replace('"', '') + '.xlsx'

# if file already exists
if os.path.exists(fileDest+'/'+output_file_name):
    print("there is a file with the same name")
    # move the old file to a different folder
    shutil.move(fileDest+'/'+output_file_name, fileDest_backup+'/'+output_file_name)
    
    # add date and time to the old filename
    date = str(datetime.today().strftime("%m.%d.%y"))
    time = str(datetime.now().strftime("%H.%M.%S"))
    new_name = 'EconLit_' + keyword.replace('"', '') + '_' + date + '_' + time + '.xlsx'
    old_file = os.path.join(fileDest_backup, output_file_name)
    new_file = os.path.join(fileDest_backup, new_name)
    os.rename(old_file, new_file)
    
# create an excel to store information
writer = pd.ExcelWriter(os.path.join(fileDest, output_file_name))
writer.save()

# columns in the excel
year_list = [] # year
all_list = [] # number of all articles in that year
keyword_list = [] # number of articles with the keyword in that year
percentage_list = [] #storing the percentage of the articles with the keyword

# put in default values
for year in range (start_year, ending_year + 1):
    year_list.append(year)
    all_list.append(-1)
    keyword_list.append(-1)
    percentage_list.append(-1.0)
    
# update the excel. This is the default excel.    
update_output(output_file_name, year_list, all_list, keyword_list, percentage_list)

In [ ]:
# find the number of articles with the keyword

# put in the keyword
driver.find_element(By.NAME, 'GuidedSearchFormData[1].SearchTerm').send_keys(keyword)
# click search
driver.find_element(By.NAME, 'SearchBtn').click()

# store the number of articles with the keyword in the excel
for year in range(start_year, ending_year + 1):
    num = search_articles(year)
    idx = year - start_year
    keyword_list[idx] = num
    update_output(output_file_name, year_list, all_list, keyword_list, percentage_list)

In [ ]:
# find the number of all articles

# remove the keyword
driver.find_element(By.NAME, 'GuidedSearchFormData[1].SearchTerm').clear()
# click search
driver.find_element(By.NAME, 'SearchBtn').click()

# store the number of all articles in the excel
for year in range(start_year, ending_year + 1):
    num = search_articles(year)
    idx = year - start_year
    all_list[idx] = num
    update_output(output_file_name, year_list, all_list, keyword_list, percentage_list)

In [ ]:
# close the window
driver.quit()

In [ ]:
# calculate the proportion
for i in range(0, len(year_list)):
    all_article_num = all_list[i]
    keyword_article_num = keyword_list[i]
    if all_article_num >= 0 & keyword_article_num >= 0:
        # calculate percentage
        pct = (100.0 * keyword_article_num)/all_article_num
        # round to 3 decimal places
        pct = float(round(pct, 3))
        # store the number
        percentage_list[i] = pct
        
# update the excel
update_output(output_file_name, year_list, all_list, keyword_list, percentage_list)

Deprecated Code

In [ ]:
keyword = 'institutions'
year = '2021'
url = 'https://web-p-ebscohost-com.proxy-um.researchport.umd.edu/ehost/resultsadvanced'\
'?vid=3&sid=b5b065a6-68c7-487d-a014-63c3a91c0f25%40redis&bquery=' + keyword + '&bdata=J'\
'mRiPWVjbiZ0eXBlPTEmc2VhcmNoTW9kZT1BbmQmc2l0ZT1laG9zdC1saXZl'
driver = get_web_driver()
driver.implicitly_wait(60)
driver.get(url)
driver.implicitly_wait(60)
# ctl00$ctl00$Column1$Column1$ctl00$ctrlResultsDualSliderDate$txtFilterDateFrom
input_start_year = driver.find_element(By.NAME, 'ctl00$ctl00$Column1$Column1$ctl00$ctrlResultsDualSliderDate$txtFilterDateFrom')
input_start_year.send_keys(year)
driver.implicitly_wait(60)
input_end_year = driver.find_element(By.NAME, 'ctl00$ctl00$Column1$Column1$ctl00$ctrlResultsDualSliderDate$txtFilterDateTo')
btn = driver.find_element(By.NAME, 'SearchBtn')
driver.implicitly_wait(60)
btn.click()
driver.quit()

In [ ]:
url = 'https://www.tripadvisor.com/'
driver = get_web_driver()
driver.implicitly_wait(60)
driver.get(url)
driver.implicitly_wait(60)
# qjfqs _G B- z _J Cj R0
element = driver.find_element(By.NAME, 'q')
print(element.text)
element.send_keys("Los Angeles")

driver.quit()